In [1]:
cd ..

/home/alberto/Work/course_interpretability_deep_learning


# Multi-omics stratification on PDAC patients

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
import optuna
import time
import dill
import shutil
from optuna.samplers import TPESampler

from src import settings
from utils import RemoveFeaturesWithZeros, RemoveFeaturesWithNaN, FeatureSelectionNMF, RemoveCorrelatedFeatures, RemoveFeaturesLowMAE
from optimization import Optimization


## Load dataset

In [3]:
methylation_data = pd.read_csv(settings.methylation_data_path, sep=";", index_col=0, decimal=",")
methylation_data.columns = methylation_data.columns.str.replace(".", "-")
methylation_data = methylation_data.T
methylation_data = methylation_data.astype(np.float32)
print("methylation_data.shape", methylation_data.shape)
methylation_data.head()

methylation_data.shape (153, 301195)


,cg00000029,cg00000236,cg00000289,cg00000292,cg00000321,cg00000622,cg00000658,cg00000714,cg00000721,cg00000734,...,ch.9.2262725R,ch.9.2285199R,ch.9.2298007R,ch.9.2473665R,ch.9.357218F,ch.9.377428R,ch.9.691424R,ch.9.837340R,ch.9.898515R,ch.9.991104F
TCGA-2J-AAB6,0.157951,0.836226,0.710511,0.560780,0.239194,0.016433,0.864604,0.087681,0.938775,0.061008,...,0.103136,0.053757,0.032478,NaN,0.064965,0.049776,0.115268,0.095954,0.084203,NaN
TCGA-2J-AAB8,0.300754,0.782242,0.574296,0.670286,0.424310,0.014747,0.885958,0.112524,0.930765,0.037198,...,0.028180,0.054483,0.022736,NaN,0.060835,0.036434,0.160082,0.059216,0.065342,0.166304
TCGA-2J-AAB9,0.257807,0.846522,0.534748,0.688073,0.295597,0.014649,0.895039,0.167297,0.940112,0.058407,...,0.059313,0.063187,0.032581,NaN,0.055342,0.069086,0.128546,0.120015,0.074940,NaN
TCGA-2J-AABA,0.239086,0.789457,0.474723,0.705372,0.530321,0.016919,0.884874,0.129581,0.910885,0.062167,...,0.122677,0.056068,0.023190,0.109351,0.056015,0.053238,0.082979,0.057172,0.045781,0.121676
TCGA-2J-AABE,0.168622,0.841684,0.591205,0.623799,0.322576,0.014408,0.898202,0.125415,0.941153,0.059365,...,0.046699,0.049177,0.032707,NaN,0.075854,0.062602,0.122072,0.082753,0.071240,NaN


In [4]:
rnaseq_data = pd.read_csv(settings.rnaseq_data_path, sep=";", index_col=0, decimal=",")
rnaseq_data = rnaseq_data.T
rnaseq_data = rnaseq_data.astype(np.float32)
print("rnaseq_data.shape", rnaseq_data.shape)
rnaseq_data.head()

rnaseq_data.shape (147, 20501)


,A1BG,A1CF,A2BP1,A2LD1,A2ML1,A2M,A4GALT,A4GNT,AAA1,AAAS,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR
TCGA-2J-AAB6,82.549698,8.187100,0.0000,163.122803,1815.789551,8517.444336,1121.052612,1.169600,1.1696,834.502930,...,14.619900,269.005798,1053.216431,0.5848,683.625671,11696.491211,869.005798,601.754395,26.315800,0.0000
TCGA-2J-AAB8,56.930698,33.842499,0.0000,185.814301,16.921301,14413.913086,392.949493,9.400700,0.9401,801.880127,...,35.722698,356.286713,829.142212,3.7603,680.611023,5829.377441,828.202087,609.165710,85.546402,0.0000
TCGA-2J-AAB9,105.787804,21.436199,1.0718,166.709503,642.015015,24311.779297,1125.401855,50.375099,0.0000,862.808105,...,57.877800,381.564789,936.763123,1.0718,646.302307,8094.319336,1083.601318,573.419128,30.010700,0.0000
TCGA-2J-AABA,99.345497,18.788200,0.0000,99.276703,873.649597,10302.006836,633.161072,6.262700,18.7882,623.767029,...,52.606899,293.721588,1511.820923,1.2525,945.670898,4829.810547,1364.646851,793.486816,31.313601,0.6263
TCGA-2J-AABE,79.401901,3.083100,0.0000,134.564499,74.610802,11076.861328,710.343811,35.147202,0.0000,702.327698,...,56.728802,431.632507,1069.215454,0.6166,564.205322,7464.775879,832.434082,468.629608,48.096199,0.0000


In [5]:
samples = methylation_data.index.intersection(rnaseq_data.index)
methylation_data = methylation_data.loc[samples]
rnaseq_data = rnaseq_data.loc[samples]
assert methylation_data.index.equals(rnaseq_data.index)
Xs= [rnaseq_data, methylation_data]
print("common samples:", len(samples))

common samples: 147


In [6]:
rnaseq_pipeline = make_pipeline(
    RemoveFeaturesWithZeros(threshold= 0.2, verbose= True),
    RemoveFeaturesLowMAE(percentage_to_keep= 0.5, verbose= True),
    RemoveCorrelatedFeatures(threshold = 0.85, verbose= True),
    FunctionTransformer(lambda x: np.log2(1 + x)),
    FeatureSelectionNMF(nmf = NMF(n_components= 512, max_iter=10000, random_state=settings.RANDOM_STATE), verbose= True),
    StandardScaler().set_output(transform= 'pandas'),
)
rnaseq_pipeline

Pipeline(steps=[('removefeatureswithzeros',
                 RemoveFeaturesWithZeros(verbose=True)),
                ('removefeatureslowmae',
                 RemoveFeaturesLowMAE(percentage_to_keep=0.5, verbose=True)),
                ('removecorrelatedfeatures',
                 RemoveCorrelatedFeatures(threshold=0.85, verbose=True)),
                ('functiontransformer',
                 FunctionTransformer(func=<function <lambda> at 0x7f7ed30c71c0>)),
                ('featureselectionnmf',
                 FeatureSelectionNMF(nmf=NMF(max_iter=10000, n_components=512,
                                             random_state=42),
                                     verbose=True)),
                ('standardscaler', StandardScaler())])

In [7]:
methylation_pipeline = make_pipeline(
    RemoveFeaturesWithNaN(threshold = 0.2, verbose= True),
    RemoveFeaturesLowMAE(percentage_to_keep= 0.1, verbose= True),
    # RemoveCorrelatedFeatures(threshold = 0.85, verbose= True),
    SimpleImputer(strategy= "mean").set_output(transform= 'pandas'),
    FeatureSelectionNMF(nmf = NMF(n_components= 256, max_iter=10000, random_state=settings.RANDOM_STATE), verbose= True),
    StandardScaler().set_output(transform= 'pandas'),
)
methylation_pipeline

Pipeline(steps=[('removefeatureswithnan', RemoveFeaturesWithNaN(verbose=True)),
                ('removefeatureslowmae',
                 RemoveFeaturesLowMAE(percentage_to_keep=0.1, verbose=True)),
                ('simpleimputer', SimpleImputer()),
                ('featureselectionnmf',
                 FeatureSelectionNMF(nmf=NMF(max_iter=10000, n_components=256,
                                             random_state=42),
                                     verbose=True)),
                ('standardscaler', StandardScaler())])

In [8]:
new_study = False
if new_study:
    shutil.rmtree("tensorboard/", ignore_errors= True)
    date = time.strftime('%Y%m%d%H')
    optimization_study = optuna.create_study(direction="maximize", sampler=TPESampler(seed = settings.RANDOM_STATE, multivariate = True, n_startup_trials = 500))
    save_pipelines = True
    n_trials = 1
    for file in os.listdir(settings.optimization_path):
        try:
            os.remove(os.path.join(settings.optimization_path, file))
        except IsADirectoryError:
            shutil.rmtree(os.path.join(settings.optimization_path, file), ignore_errors= True)
else:
    date = "2023062618"
    with open(os.path.join(settings.optimization_path, f'optimization_optuna_{date}.pkl'), 'rb') as file:
        optimization_study = dill.load(file)
    save_pipelines = False
    n_trials = 1000

In [9]:
date = time.strftime('%Y%m%d%H')
optimization_study = optuna.create_study(direction="maximize", sampler=TPESampler(seed = settings.RANDOM_STATE, multivariate = True, n_startup_trials = 500, 
                                                                                  warn_independent_sampling=False))

/home/alberto/anaconda3/envs/course_interpretability_dl/lib/python3.10/site-packages/optuna/samplers/_tpe/sampler.py:282: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2023-07-02 00:50:10,381] A new study created in memory with name: no-name-e408157f-b338-465f-847b-72bcd6f866c3


In [ ]:
pipelines= [rnaseq_pipeline, methylation_pipeline]
func_objective = lambda trial: Optimization().objective(trial= trial, Xs= Xs, samples= samples, pipelines= pipelines, 
                                                        features_per_component_options= [1, 9, 2], num_layers_option= [1,2,1], num_units_option= [2,20, 2],
                                                        n_epochs_option= [20,100,20], lambda_option = [0.001, 1, 0.25], n_clusters_option= [2,6,1],
                                                        latent_space_option = [50, 150, 50], batch_size=32,
                                                        random_state=settings.RANDOM_STATE, n_jobs= 2, save_pipelines= save_pipelines,
                                                        folder= settings.optimization_path)

keep_trying = True
while keep_trying:
    try:
        optimization_study = Optimization.optimize_optuna_and_save(study= optimization_study, n_trials = n_trials, date=date,
                                                                   show_progress_bar= True, folder= settings.optimization_path, func= func_objective)
        if new_study:
            keep_trying = False
    except FileNotFoundError:
        pass
    except ValueError:
        pass

  0%|          | 0/1000 [00:00<?, ?it/s]

[I 2023-07-02 00:50:10,396] Trial 0 pruned. 
[I 2023-07-02 00:50:10,403] Trial 1 pruned. 
[I 2023-07-02 00:50:10,407] Trial 2 pruned. 
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/alberto/anaconda3/envs/course_interpretability_dl/lib/python3.10/site-packages/pytorch_lightning/loops/utilities.py:70: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
/home/alberto/anaconda3/envs/course_interpretability_dl/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
/home/alberto/anaconda3/env

In [ ]:
optimization_study.best_params

In [ ]:
fig = optuna.visualization.plot_optimization_history(optimization_study)
fig.show()

In [ ]:
fig = optuna.visualization.plot_param_importances(optimization_study)
fig.show()

In [ ]:
fig = optuna.visualization.plot_slice(optimization_study)
fig.show()

In [ ]:
optimization_results = pd.read_csv(os.path.join(settings.optimization_path, f"optimization_results_{date}.csv"))
best_trial = optimization_results.iloc[0]
print("optimization_results.shape", optimization_results.shape)
optimization_results.head()